In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict, Counter
from functools import reduce
import seaborn as sns
from scipy.stats import binned_statistic
from matplotlib.lines import Line2D

In [2]:
%cd ../..

/nfs/homedirs/fuchsgru/MastersThesis


In [3]:
import data.constants as dc

In [4]:
collection_name = 'week20_laplace_all_datasets'
collection = seml.database.get_collection(collection_name)

In [5]:
by_status = defaultdict(set)
for r in collection.find():
    by_status[r['status']].add(r['config']['data']['dataset'])

In [6]:
by_status

defaultdict(set,
            {'COMPLETED': {'amazon_photo',
              'citeseer',
              'coauthor_cs',
              'cora_full',
              'pubmed'},
             'KILLED': {'ogbn_arxiv'},
             'PENDING': {'ogbn_arxiv'}})

In [7]:
failed_exps = [r for r in collection.find() if r['status'] == 'FAILED']

In [8]:
killed_exps = [r for r in collection.find() if r['status'] == 'KILLED']

In [9]:
killed_exps[0]

{'_id': 526,
 'batch_id': 7,
 'status': 'KILLED',
 'seml': {'executable': 'training_semi_supervised_node_classification.py',
  'name': 'week20_dropout_all_datasets',
  'output_dir': '/nfs/students/fuchsgru/seml_output/week20',
  'working_dir': '/nfs/homedirs/fuchsgru/MastersThesis',
  'conda_environment': 'base',
  'source_files': [['model/prediction.py',
    ObjectId('6241cea033a2eea9a7faeaba')],
   ['data/split.py', ObjectId('6241cea033a2eea9a7faeabc')],
   ['model/build.py', ObjectId('6241cea033a2eea9a7faeabe')],
   ['evaluation/pipeline/base.py', ObjectId('6241cea033a2eea9a7faeac0')],
   ['data/__init__.py', ObjectId('6241cea033a2eea9a7faeac2')],
   ['evaluation/lipschitz.py', ObjectId('6241cea033a2eea9a7faeac4')],
   ['model/parametrization.py', ObjectId('6241cea033a2eea9a7faeac6')],
   ['model/constants.py', ObjectId('6241cea033a2eea9a7faeac8')],
   ['evaluation/__init__.py', ObjectId('6241cea033a2eea9a7faeaca')],
   ['data/util.py', ObjectId('6241cea033a2eea9a7faeacc')],
   ['ev

In [10]:
set(ex)

NameError: name 'ex' is not defined

In [ ]:
print(''.join(failed_exps[0]['fail_trace']))

In [ ]:
print(''.join(failed_exps[0]['captured_out']))

In [11]:
failed_exps[0].keys()

IndexError: list index out of range

In [12]:

collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    # print(ex['result'].keys())
    ex['metrics'] = ex['result']['results']

In [13]:
METRIC = 'Metric'
OOD_AUROC = 'OOD AUC-ROC'
OOD_AUCPR = 'OOD AUC-PR'
MISCLASSICIFACTION_AUROC = 'Misclassification AUC-ROC'
MISCLASSICIFACTION_AUCPR = 'Misclassification AUC-PR'

DATASET = 'Dataset'

SETTING = 'Setting'
HYBRID = 'Inductive'
TRANSDUCTIVE = 'Transductive'
LOC = 'Leave Out Classes'
NORMAL = 'Normal'
BERNOULLI = 'Bernoulli'
NO_EDGES = 'No Edges'
PROXY = 'Proxy'
EXPERIMENT = 'Experiment'

OOD_TYPE = 'OOD Type'


RESIDUAL = 'Residual'
SPECTRAL_NORM = 'Spectral Norm'
WEIGHT_SCALE = 'Weight Scale'
LOWER_LIPSCHITZ = 'Empirical Lower Lipschitz Bound'
UPPER_LIPSCHITZ = 'Empirical Upper Lipschitz Bound'



EPISTEMIC = 'Epistemic'
ALEATORIC = 'Aleatoric'

ACCURACY = 'Accuracy'
ACCURACY_ID = 'In-distribution Accuracy'
ECE = 'Expected Calibration Error'

MODEL = 'Model'
DROPOUT = 'Dropout'
DROP_EDGE = 'Drop Edge'

In [14]:
mode = 'test'

In [15]:
data_proxy, data_acc_ece = [], []
for ex in experiments:
    cfg = ex['config']
    base = {
        SETTING : {dc.HYBRID : HYBRID, dc.TRANSDUCTIVE : TRANSDUCTIVE}[cfg['data']['setting']],
        DATASET : cfg['data']['dataset'],
    }
    data_acc_ece.append(base | {
        OOD_TYPE : cfg['data']['ood_type'],
        ACCURACY : ex['metrics'][f'accuracy_{mode}_{mode}'][0],
        ECE : ex['metrics'][f'ece_{mode}_{mode}'][0]['value'],
    })
    if cfg['data']['ood_type'] == dc.PERTURBATION:
        ood_types = (
            (BERNOULLI, 'ber'),
            (NORMAL, 'normal'),
        )
    elif cfg['data']['ood_type'] == dc.LEFT_OUT_CLASSES:
        ood_types = (
            (LOC, 'loc'),
        )
    else:
        raise ValueError(cfg['data']['ood_type'])
    
    for no_edge_suffix, no_edges in (('-no-edges', True), ('', False)):
    
        for ood_type, ood_name in ood_types:
            for proxy, proxy_name in ((ALEATORIC, 'expected-softmax-entropy'), (EPISTEMIC, 'mutual-information')):
                data_proxy += [
                    base | {
                        EXPERIMENT : ood_type,
                        PROXY : proxy,
                        OOD_AUROC : ex['metrics'][f'ood_auroc_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        OOD_AUCPR : ex['metrics'][f'ood_aucpr_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        MISCLASSICIFACTION_AUROC : ex['metrics'][f'misclassification_auroc_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        MISCLASSICIFACTION_AUCPR : ex['metrics'][f'misclassification_aucpr_{proxy_name}_{ood_name}{no_edge_suffix}_{mode}'][0]['value'],
                        NO_EDGES : no_edges,
                    }
                ]
    
        
        
    
df_proxy = pd.DataFrame(data_proxy)
df_acc_ece = pd.DataFrame(data_acc_ece)

In [16]:
df_acc_ece

,Setting,Dataset,OOD Type,Accuracy,Expected Calibration Error
0,Transductive,cora_full,left-out-classes,0.832031,0.568850
1,Transductive,cora_full,left-out-classes,0.758893,0.517596
2,Transductive,cora_full,left-out-classes,0.812379,0.553822
3,Transductive,cora_full,left-out-classes,0.792157,0.533692
4,Transductive,cora_full,left-out-classes,0.816092,0.555870
...,...,...,...,...,...
495,Inductive,amazon_photo,perturbations,0.882742,0.638955
496,Inductive,amazon_photo,perturbations,0.919368,0.665100
497,Inductive,amazon_photo,perturbations,0.907268,0.660601
498,Inductive,amazon_photo,perturbations,0.886855,0.639863


In [17]:
def agg_mean_and_std(group):
    mean = group.mean()
    std = group.std()
    return f'{mean:.2f} ± {std:.2f}'
    return mean

df_proxy_agg = df_proxy.groupby([SETTING, DATASET, EXPERIMENT, NO_EDGES, PROXY]).agg(agg_mean_and_std)

In [18]:
df_tmp = df_proxy_agg.reset_index()
df_tmp = df_tmp.melt(id_vars=[SETTING, DATASET, EXPERIMENT, NO_EDGES, PROXY], value_vars = [OOD_AUROC, OOD_AUCPR, MISCLASSICIFACTION_AUROC, MISCLASSICIFACTION_AUCPR], var_name='Metric')
df_tmp = df_tmp[(df_tmp[METRIC] == OOD_AUROC) | (df_tmp[METRIC] == OOD_AUCPR)]
df_tmp = df_tmp.pivot(index=[SETTING, DATASET, PROXY], columns=[METRIC, EXPERIMENT, NO_EDGES]).T.sort_index().T
df_tmp.to_csv('~/laplace_all_datasets_ood_detection.csv')

In [19]:
df_tmp

value               \
Metric                                OOD AUC-PR                
Experiment                             Bernoulli                
No Edges                                   False         True   
Setting      Dataset      Proxy                                 
Inductive    amazon_photo Aleatoric  0.50 ± 0.02  0.80 ± 0.04   
                          Epistemic  0.50 ± 0.02  0.75 ± 0.04   
             citeseer     Aleatoric  0.76 ± 0.01  0.94 ± 0.03   
                          Epistemic  0.78 ± 0.04  0.86 ± 0.03   
             coauthor_cs  Aleatoric  0.59 ± 0.02  1.00 ± 0.00   
                          Epistemic  0.59 ± 0.02  0.87 ± 0.02   
             cora_full    Aleatoric  0.61 ± 0.03  0.95 ± 0.03   
                          Epistemic  0.61 ± 0.03  0.83 ± 0.03   
             pubmed       Aleatoric  0.68 ± 0.02  0.78 ± 0.06   
                          Epistemic  0.73 ± 0.04  0.86 ± 0.05   
Transductive amazon_photo Aleatoric  0.51 ± 0.02  0.78 ± 0.06   
                          Epistemic  0.50 ± 0.03  0.73 ± 0.06   
             citeseer     Aleatoric  0.69 ± 0.06  0.97 ± 0.02   
                          Epistemic  0.63 ± 0.05  0.75 ± 0.05   
             coauthor_cs  Aleatoric  0.56 ± 0.02  1.00 ± 0.01   
                          Epistemic  0.55 ± 0.02  0.87 ± 0.01   
             cora_full    Aleatoric  0.57 ± 0.03  0.94 ± 0.04   
                          Epistemic  0.55 ± 0.03  0.82 ± 0.04   
             pubmed       Aleatoric  0.57 ± 0.03  0.73 ± 0.12   
                          Epistemic  0.58 ± 0.02  0.76 ± 0.11   

                                                                    \
Metric                                                               
Experiment                          Leave Out Classes                
No Edges                                        False         True   
Setting      Dataset      Proxy                                      
Inductive    amazon_photo Aleatoric       0.71 ± 0.07  0.52 ± 0.05   
                          Epistemic       0.72 ± 0.07  0.47 ± 0.04   
             citeseer     Aleatoric       0.61 ± 0.11  0.47 ± 0.08   
                          Epistemic       0.44 ± 0.11  0.38 ± 0.06   
             coauthor_cs  Aleatoric       0.83 ± 0.09  0.77 ± 0.06   
                          Epistemic       0.67 ± 0.05  0.65 ± 0.06   
             cora_full    Aleatoric       0.84 ± 0.06  0.69 ± 0.05   
                          Epistemic       0.58 ± 0.05  0.57 ± 0.05   
             pubmed       Aleatoric       0.23 ± 0.02  0.25 ± 0.04   
                          Epistemic       0.23 ± 0.02  0.24 ± 0.02   
Transductive amazon_photo Aleatoric       0.65 ± 0.09  0.58 ± 0.03   
                          Epistemic       0.67 ± 0.09  0.51 ± 0.03   
             citeseer     Aleatoric       0.59 ± 0.17  0.43 ± 0.17   
                          Epistemic       0.49 ± 0.16  0.40 ± 0.15   
             coauthor_cs  Aleatoric       0.66 ± 0.16  0.64 ± 0.09   
                          Epistemic       0.58 ± 0.13  0.54 ± 0.10   
             cora_full    Aleatoric       0.87 ± 0.04  0.75 ± 0.06   
                          Epistemic       0.59 ± 0.08  0.63 ± 0.09   
             pubmed       Aleatoric       0.24 ± 0.09  0.23 ± 0.08   
                          Epistemic       0.24 ± 0.10  0.21 ± 0.07   

                                                                            \
Metric                                                         OOD AUC-ROC   
Experiment                                Normal                 Bernoulli   
No Edges                                   False         True        False   
Setting      Dataset      Proxy                                              
Inductive    amazon_photo Aleatoric  0.47 ± 0.04  0.33 ± 0.02  0.51 ± 0.02   
                          Epistemic  0.49 ± 0.03  0.37 ± 0.02  0.51 ± 0.02   
             citeseer     Aleatoric  0.57 ± 0.03  0.51 ± 0.02  0.53 ± 0.03   
                          Epistemic  0.73 

In [20]:
df_tmp = df_proxy_agg.reset_index()
df_tmp = df_tmp.melt(id_vars=[SETTING, DATASET, EXPERIMENT, NO_EDGES, PROXY], value_vars = [OOD_AUROC, OOD_AUCPR, MISCLASSICIFACTION_AUROC, MISCLASSICIFACTION_AUCPR], var_name='Metric')
df_tmp = df_tmp[(df_tmp[METRIC] == MISCLASSICIFACTION_AUROC) | (df_tmp[METRIC] == MISCLASSICIFACTION_AUCPR)]
df_tmp = df_tmp.pivot(index=[SETTING, DATASET, PROXY], columns=[METRIC, EXPERIMENT, NO_EDGES]).T.sort_index().T
df_tmp.to_csv('~/laplace_all_datasets_misclassification_detection.csv')